In [1]:
import pandas as pd 
import os
from tqdm import tqdm

In [2]:
def split_xyz(file_xyz, write_root, db_name="radqm9_test"):
    """ intake xyz file and split it into separate files """
    if not os.path.exists(write_root):
        os.makedirs(write_root)

    with open(file_xyz, 'r') as f:
        data = f.readlines()
    
    
    file_counter = 0 
    temp_block = []
    for i in range(len(data)-1): 
        # check if next line is a number
        if(data[i+1].split()[0].isdigit()):
            write_file = open('{}/{}_{}.xyz'.format(write_root, db_name, file_counter), 'w')
            write_file.writelines(temp_block)
            write_file.close()
            file_counter += 1
            temp_block = []
        else: 
            temp_block.append(data[i])

    write_file = open('{}/{}_{}.xyz'.format(write_root, db_name, file_counter), 'w')
    write_file.writelines(temp_block)
    write_file.close()
    file_counter += 1
    print(f"Total files created: {file_counter}")

    

split_xyz(
    file_xyz='/home/santiagovargas/Downloads/rad_qm9_train.xyz',
    write_root='/home/santiagovargas/Downloads/radQM9_xyz_train', 
    db_name='radqm9_train'
    )



Total files created: 876951


In [4]:
file = "/home/santiagovargas/Downloads/radQM9_xyz_train/radqm9_train_0.xyz"
with open(file, 'r') as f:
    data = f.readlines()
    print(data[1])

# todos
# grab mol id 
# grab position type 
# grab partial spin
# grab charge
# grab spin
# grab energy

Properties=species:S:1:pos:R:3:forces:R:3 energy=-352.6011882796 charge=0 spin=1 partial_charge="_JSON [[[0.015858], [0.273335], [-0.190021], [0.231293], [-0.190459], [0.273762], [0.016057], [-0.023211], [0.014057], [-0.411295], [0.013833], [-0.023209]], [[0.022974], [0.282245], [-0.18833], [0.236445], [-0.188693], [0.282547], [0.023046], [-0.032857], [-0.000179], [-0.404471], [-4.1e-05], [-0.032687]]]" partial_spin="_JSON [[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]], [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]]" mol_id=3883 position_type=start pbc="F F F"



In [62]:
test = "/home/santiagovargas/Downloads/20240123_radqm9_sp_engrad/engrads/radqm9_sp_0.json"
df = pd.read_json(test)

def parse_json_radqm9(file, solvent="vacuum"):
    df = pd.read_json(file)
    df = df[df['solvent'] == solvent]
    species = df['species'].tolist()
    # sort species
    species_sorted = [i.sort() for i in species]
    return species_sorted

def parse_species_radqm9(file, solvent="vacuum"):
    df = pd.read_json(file)
    df = df[df['solvent'] == solvent]
    species = df['species'].tolist()
    # sort species
    [i.sort() for i in species]
    #print(species)
    # convert to strings
    species_sorted = ["".join(i) for i in species]
    df["species_sorted"] = species_sorted
    #overwrite json 
    df.to_json(file)
    # convert to set to remove duplicates
    species_sorted = list(set(species_sorted))
    return species_sorted

root = "/home/santiagovargas/Downloads/20240123_radqm9_sp_engrad/engrads"
comp_species = []
for file in tqdm(os.listdir(root)):
    if file.endswith(".json"):
        #print(os.path.join(root, file))
        comp_species.append(parse_species_radqm9(os.path.join(root, file)))

# flatten list, convert to set to remove duplicates, convert back to list
comp_species = list(set([item for sublist in comp_species for item in sublist]))

100%|██████████| 3/3 [00:28<00:00,  9.50s/it]


In [68]:
test = "/home/santiagovargas/Downloads/rad_qm9_test.xyz"
# read first 10 lines
with open(test, 'r') as reader:
    for i in range(10):
        print(reader.readline())

14

Properties=species:S:1:pos:R:3:forces:R:3 energy=-320.1603647112 charge=1 spin=2 partial_charge="_JSON [[[0.521985], [-0.020495], [0.07576], [-0.0813], [0.37161], [-0.143809], [0.414569], [-0.011793], [0.020345], [0.05602], [0.141687], [0.031641], [-0.315973], [-0.060248]], [[0.512205], [-0.044511], [-0.021262], [-0.042619], [0.297168], [-0.110957], [0.522845], [-0.008483], [0.023654], [0.060323], [0.149186], [0.048673], [-0.286253], [-0.09997]], [[0.5111], [-0.033722], [-0.079651], [-0.052746], [0.30909], [-0.118045], [0.518911], [-0.029334], [0.028164], [0.062032], [0.145735], [0.117956], [-0.280639], [-0.09885]]]" partial_spin="_JSON [[[0.063129], [-0.056011], [0.429004], [-0.053889], [0.402271], [-0.121796], [0.341267], [-0.004376], [-0.005204], [0.00746], [0.015858], [0.012937], [-0.013862], [-0.016788]], [[0.068125], [-0.058936], [0.472648], [-0.007047], [0.369011], [-0.090373], [0.247894], [-0.005177], [-0.006603], [0.010034], [0.014656], [0.013101], [-0.013246], [-0.014088]

In [64]:
comp_species

['CCCCHHHHNNO',
 'CCCCCCCCCHHHHHHHHHHHHHHHHHH',
 'CCCCHHHHHHNNNNO',
 'CCCCHHHHHHNNOOO',
 'CCCCCCCHHHHHHHHHHHN',
 'CCCCCCHHHHHHHHOO',
 'CCCCHHHHHHHHNNNNO',
 'CCCCHHHHHNNNOO',
 'CCCCCCCCHHHHHHHN',
 'CCCCCCCHHHHHHHHHHHHHNO',
 'CCCCCCHHHHHHHHHNNN',
 'CCCCCCCCHHHHHHHHHHHHHHHHHN',
 'CCCCCCCCHHHHHN',
 'CCCCCHHHHHHHHHNOOO',
 'CCCCCCCCHHHHHHHHHHO',
 'CCCCCHHHHHHHHHHHNOOO',
 'CCCCCCCHHHHHHHHHNO',
 'CCCCCCCHHHHHHHHHHHHHHNN',
 'CCCCCCCHHHHHHHHHN',
 'CCCHHHHHHHNNNNN',
 'CCCCCCHHHHHHHHHHHHHHNNO',
 'CCCCCHHHHHHHHHHHNNNO',
 'CCHHHNNNNNO',
 'CCCCCCCHHHHHHHHHHHHHHHHO',
 'CCCCHHNNNN',
 'CCCFHNNOO',
 'CCCCHHHHHHHHHNOOO',
 'CCCCCHHHHHHHHHHNNO',
 'CCCCCHHHHHHNNNN',
 'CCCCCCCHHHHHHHHHH',
 'CCCCCHHHHHHHHOOOO',
 'CCCCHHHHHHHNOOO',
 'CCCHNNNOO',
 'CCCCCCCCHHHHHHHHHHHHHHHHHHO',
 'CCCCCHHHHHHHHHNO',
 'CCCHHNNNNO',
 'CCCCCHHHHHHHHHHHHOO',
 'CCCCCCHHHHHHHNNN',
 'CCCCCCCCHHHHHHHHHHHHHHHHO',
 'CCCCCCCCCHHHHHHHHHHHHHHHH',
 'CCCCCCFHHHHHO',
 'CCCFHHNNNO',
 'CCCCCHHHHHHHNNNO',
 'CCCCHHHHNNOO',
 'CCCCCHHHHNNNN',
 'CCCCCC